In [19]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# /kaggle/input/plant-pathology-2020-fgvc7/images/Test_1743.jpg
# /kaggle/input/plant-pathology-2020-fgvc7/images/Train_1524.jpg

In [20]:
import numpy as np
import pandas as pd
import os


test_df = pd.read_csv("../input/plant-pathology-2020-fgvc7/test.csv")
train_df = pd.read_csv("../input/plant-pathology-2020-fgvc7/train.csv")

In [21]:
train_df.head()

In [22]:
# healthy, multiple_diseases, rust, scab 컬럼이 one-hot encoding 형식으로 되어있음
# 검증: healthy, multiple_diseases, rust, scab 컬럼을 합해서 sum을 만들고
# sum이 1보다 큰지, 아니면 0인지 확인

train_df['sum'] = train_df['healthy'] + train_df['multiple_diseases'] + train_df['rust'] + train_df['scab']
train_df[(train_df['sum'] > 1) | (train_df['sum'] == 0)]

In [23]:
# 이미지의 절대 경로를 DataFrame에 추가하고, 개별 컬럼 별 0/1 값을 구분하여 클래스 라벨로 생성

pd.set_option('max_colwidth', 100)

IMAGE_DIR = '/kaggle/input/plant-pathology-2020-fgvc7/images'
train_df['path'] = IMAGE_DIR + '/' + train_df['image_id'] + '.jpg'

train_df.head()


In [24]:
# 이미지의 label을 DataFrame에 추가

def get_label(x):
    if x['healthy'] == 1:
        return 'healthy'
    elif x['multiple_diseases'] == 1:
        return 'multiple_diseases'
    elif x['scab'] == 1:
        return 'scab'
    elif x['rust'] == 1:
        return 'rust'
    else:
        return 'None'

train_df['label'] = train_df.apply(lambda x: get_label(x), axis=1)
train_df.head()

In [25]:
# 학습 이미지 건수 및 label 별 건수 확인
print('train shape: ', train_df.shape)
print()
print('label 별 건수')
train_df['label'].value_counts()

In [26]:
# 원본 이미지 시각화

# 녹병균(Rust), 박테리아성 질환(scab), 복합질병(multiple_diseases), 건강(healthy)
# 이미지 size는 (1365, 2048)

import seaborn as sns
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

def show_grid_images(image_path_list, augmentor=None, ncols=4, title=None):
    figure, axs = plt.subplots(figsize=(22, 4), nrows=1, ncols=ncols)
    for i in range(ncols):
        image = cv2.cvtColor(cv2.imread(image_path_list[i]), cv2.COLOR_BGR2RGB)
        if augmentor is not None:
            image = augmentor(image=image)['image']
        axs[i].imshow(image)
        axs[i].set_title(title)
        print(image.shape)

rust_image_list = train_df[train_df['label'] == 'rust']['path'].iloc[:6].tolist()
scab_image_list = train_df[train_df['label'] == 'scab']['path'].iloc[:6].tolist()
healthy_image_list = train_df[train_df['label'] == 'healthy']['path'].iloc[:6].tolist()
multiple_image_list = train_df[train_df['label'] == 'multiple_diseases']['path'].iloc[:6].tolist()

show_grid_images(rust_image_list, ncols=6, title='rust')
show_grid_images(scab_image_list, ncols=6, title='scab')
show_grid_images(healthy_image_list, ncols=6, title='healthy')
show_grid_images(multiple_image_list, ncols=6, title='multiple')

In [28]:
# 이미지 Augmentation 적용

# cutout과 같은 noise는 나뭇잎의 병균 반점과 헷갈릴 수 있으므로 사용하지 않음
# 전체 이미지가 초록색 계열이고 병균 반점이 특정 색깔을 가지고 있으므로, 색상의 변화는 적용하지 않음
# 전반적으로 판별하려는 나뭇잎이 전체 이미지의 중앙에 있으므로, scale등의 적용을 고려

import albumentations as A


augmentor_01 = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(scale_limit=(0.7, 0.9), p=0.5, rotate_limit=30),
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),
    A.Blur(p=0.2)
])

# Apply augmentor_01 to rust images
show_grid_images(rust_image_list, augmentor=None, ncols=6, title='original rust')
show_grid_images(rust_image_list, augmentor=augmentor_01, ncols=6, title='augmented rust')

# Apply augmentor_01 to scab images
show_grid_images(scab_image_list, augmentor=None, ncols=6, title='original scab')
show_grid_images(scab_image_list, augmentor=augmentor_01, ncols=6, title='augmented scab')

In [30]:
# Sequence 기반의 Dataset 생성

# image size의 높이와 너비가 다를 수 있을 경우를 고려하여, image_size를 튜플로 입력
# opencv의 resize()는 인자로 이미지 크기를 입력 받는데, 가로 x 세로 (너비 x 높이)의 개념으로 입력
# image array의 경우에는 행 x 열 (높이 x 너비)이므로 resize() 호출 시 이를 감안해야 함.
# kaggle competition의 test data의 결과를 submit 하므로 test data의 label이 없음
# 따라서, Dataset의 label_batch 값이 None이 될 수 있는 경우를 감안하여 코드 재수정 필요

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
import sklearn
import cv2


class Plant_Dataset(Sequence):
    def __init__(self, image_filenames, labels, image_size=(224, 224),
                batch_size=64, augmentor=None, shuffle=False, pre_func=None):
        '''
        :parameters
        image_filenames: opencv로 image를 로드할 파일의 절대 경로
        labels: 해당 image의 label
        batch_size: __getitem__(self, index) 호출 시 마다 가져올 데이터 batch 건수
        augmentor: albumentations 객체
        shuffle: 학습 데이터의 경우 epoch 종료 시 마다 데이터를 섞을 지 여부
        '''
        
        # 객체 생성 인자로 들어온 값을 객체 내부 변수로 할당
        self.image_filenames = image_filenames
        self.labels = labels
        self.image_size = image_size
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.pre_func = pre_func
        self.shuffle = shuffle
        
        # train_data의 경우, 객체 생성 시 한번 데이터를 shuffle
        if self.shuffle:
            # self.on_epoch_end()
            pass
        
    # Sequence를 상속받은 Dataset은 batch_size 단위로 입력된 데이터를 처리

    # __len__()은 전체 데이터 건수가 주어졌을 때, batch_size 단위로 몇 번 데이터를 반환하는지
    def __len__(self):
        # batch_size 단위로 데이터를 몇 번 가져와야하는지 계산하기 위해
        # 전체 데이터 건수를 batch_size로 나눈다.
        # 정수로 정확히 나눠지지 않을 경우, 1회를 더해준다.
        return int(np.ceil(len(self.image_filenames) / self.batch_size))

    # batch_size 단위로 image_array, label_array 데이터를 가져와서 변환한 뒤 다시 반환
    # 인자로 몇 번째 batch 인지 나타내는 index를 입력하면 해당 순서에 해당하는 batch_size 만틈의
    # 데이터를 가공하여 반환한다.
    # -> batch_size 개수 만큼 변환된 image_array와 label_array 반환
    def __getitem__(self, index):
        '''
        :parameters
        index: 몇 번째 batch 인지 나타냄
        '''

        # batch_size 만큼 순차적으로 데이터를 가져오기 위해서 array에서
        # index*self.batch_size:(index+1)*self.batch_size 만큼의 연속 데이터를 가져온다.
        image_name_batch = self.image_filenames[index*self.batch_size:(index+1)*self.batch_size]

        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]

        # label_batch가 None이 될 수 있음
        else:
            label_batch = None

        # 만일 객체 생성 인자로 albumentation으로 만든 augmentor가 주어진다면 아래와 같이 augmentor를 이용하여 image 변환
        # image_batch 배열은 float32로 설정
        image_batch = np.zeros(
            (image_name_batch.shape[0], self.image_size[0], self.image_size[1], 3),
            dtype='float32'
        )

        # batch_size에 담긴 건수만큼 iteration 하면서 opencv image load
        # -> image augmentation 변환 (augmentor가 not None일 경우)
        # -> image_batch에 담음
        for image_index in range(image_name_batch.shape[0]):
            image = cv2.cvtColor(cv2.imread(image_name_batch[image_index]), cv2.COLOR_BGR2RGB)
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
            # 원본 이미지와 다르게 resize 적용
            # opencv의 resize는 (가로, 세로)의 개념
            # 배열(array)은 (높이;행, 너비;열)의 개념이므로 이에 주의하여 opencv resize 인자를 입력 필요
            image = cv2.resize(image, (self.image_size[1], self.image_size[0]))
            # 만일 preprocessing_input이 pre_func 인자로 들어오면 이를 이용하여 scaling 적용
            if self.pre_func is not None:
                image = self.pre_func(image)

            image_batch[image_index] = image

        return image_batch, label_batch
    
    # epoch가 한 번 수행이 완료 될 때, 모델의 fit()에서 호출
    def on_epoch_end(self):
        if (self.shuffle):
            # 전체 image 파일의 위치와 label의 쌍을 맞춰서 섞어준다.
            # sklearn의 utils.shuffle에서 해당 기능을 제공
            self.image_filenames, self.labels = sklearn.utils.shuffle(self.image_filenames, self.labels)
        else:
            pass

In [31]:
# 학습 데이터용 DataFrame에서 학습용/검증용 이미지 절대 경로와 Label 추출하고 이를 Dataset으로 생성

# 이미 학습용 DataFrame에 'healthy', 'multiple_disseases', 'rust', 'scab' 순으로 one-hot encoding 되어 있음
# kaggle에서 테스트 데이터의 예측 겨과를 'healthy', 'multiple_diseases', 'rust', 'scab' 순서로 제출을 요구하므로,
# 이를 별도로 다시 one-hot encoding 해서는 안됨.
# Augmentation은 앞에서 생성한 augmentor_01을 적용
# pre_func는 xception용 Preprocessing 함수 적용

In [32]:
sample_df = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv')
sample_df.head()

In [33]:
from sklearn.model_selection import train_test_split


def get_train_valid(train_df, valid_size=0.2, random_state=2021):
    
    train_path = train_df['path'].values
    
    # 별도의 one-hot encoding을 하지 않고
    # 'healthy', 'multiple_diseases', 'rust', 'scab' 컬럼들을 모두 
    # Numpy array로 변환하는 수준으로 label을 one-hot encoding 적용
    train_label = train_df[['healthy', 'multiple_diseases', 'rust', 'scab']].values
    
    tr_path, val_path, tr_label, val_label = train_test_split(
                                                train_path,
                                                train_label,
                                                test_size=valid_size,
                                                random_state=random_state
                                            )
    
    print('tr_path shape: ', tr_path.shape)
    print('tr_label shape: ', tr_label.shape)
    print('val_path shape: ', val_path.shape)
    print('val_label shape: ', val_label.shape)
    
    return tr_path, val_path, tr_label, val_label
    

In [34]:
from tensorflow.keras.applications.xception import preprocess_input as xcp_preprocess_input
from tensorflow.keras.applications.efficientnet import preprocess_input as eff_preprocess_input


# image size는 224 x 224로 Dataset 생성
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 64

tr_path, val_path, tr_label, val_label = get_train_valid(train_df, valid_size=0.2, random_state = 2021)

tr_ds = Plant_Dataset(
    tr_path,
    tr_label,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    augmentor=augmentor_01, shuffle=True,
    pre_func=xcp_preprocess_input
)

val_ds = Plant_Dataset(
    val_path,
    val_label,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    augmentor=None,
    shuffle=False,
    pre_func=xcp_preprocess_input
)

tr_image_batch, tr_label_batch = next(iter(tr_ds))
val_image_batch, val_label_batch = next(iter(val_ds))

print('tr_image_batch shape: ', tr_image_batch.shape)
print('val_image_batch shape: ', val_image_batch.shape)
print('tr_label_batch shape: ', tr_label_batch.shape)
print('val_label_batch shape: ', val_label_batch.shape)
print()
print(tr_image_batch[0], val_image_batch[0])

In [35]:
# create_model() 함수 생성

# resnet50v2, xception, efficientnetb0~b7 등의 Pretrained 모델을 생성
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler

from tensorflow.keras.applications import Xception, ResNet50V2, EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7
import tensorflow as tf

def create_model(model_type='efficientnetb0', in_shape=(224, 224, 3), n_classes=4):
    input_tensor = Input(shape=in_shape)
    
    if model_type == 'resnet50v2':
        base_model = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'xception':
        base_model = tf.keras.applications.Xception(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb0':
        base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb1':
        base_model = tf.keras.applications.EfficientNetB1(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb2':
        base_model = tf.keras.applications.EfficientNetB2(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb3':
        base_model = tf.keras.applications.EfficientNetB3(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb4':
        base_model = tf.keras.applications.EfficientNetB4(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb5':
        base_model = tf.keras.applications.EfficientNetB5(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb6':
        base_model = tf.keras.applications.EfficientNetB6(include_top=False, weights='imagenet', input_tensor=input_tensor)
    elif model_type == 'efficientnetb7':
        base_model = tf.keras.applications.EfficientNetB7(include_top=False, weights='imagenet', input_tensor=input_tensor)
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    preds = Dense(units=n_classes, activation='softmax')(x)
    model = Model(inputs=input_tensor, outputs=preds)
    
    return model

In [36]:
# xception model create and train

# - inmage size = (224, 224, 4)
# - Learning Rate Scheduler: ReduceLROnPlateau -> Learning Rate: 0.0001
# - epochs: 10
# - metrics: ROC-AUC

from tensorflow.keras.metrics import AUC

xcp_model_01 = create_model(model_type='xception', in_shape=(224, 224, 3))
xcp_model_01.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=[AUC()]
)

# 3번의 iteration 내에 validation loss가 향상되지 않으면 learning rate를 기존 learning rate * 0.2로 감소
rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    mode='min',
    verbose=1
)

# 10번의 iteration 내에 validation loss가 향상되지 않으면 더 이상 학습하지 않고 종료
ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min',
    verbose=1
)

history = xcp_model_01.fit(
    tr_ds,
    epochs=10,
    steps_per_epoch=int(np.ceil(tr_path.shape[0]/BATCH_SIZE)),
    validation_data=val_ds,
    validation_steps=int(np.ceil(val_path.shape[0]/BATCH_SIZE)),
    callbacks=([rlr_cb, ely_cb]),
    verbose=1
)

In [37]:
# 테스트 데이터로 Plant의 질병을 예측하고 Kaggle에 제출할 submit.csv 파일 만들기

# - 테스트용 DataFrame에 이미지 경로 추가
# - 테스트용 Dataset 생성. label은 테스트 데이터에서 알 수 없으므로 None으로 입력

sample_df = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/sample_submission.csv')
sample_df.head()

In [38]:
IMAGE_DIR = '/kaggle/input/plant-pathology-2020-fgvc7/images'
# test_df = pd.read_csv("../input/plant-pathology-2020-fgvc7/test.csv")
test_df['path'] = IMAGE_DIR + "/" + test_df['image_id'] + '.jpg'

test_df.head(10)

In [41]:
# 테스트용 Dataset을 생성하고, 이를 이용하여 model의 predict()를 호출하여 이미지 예측을 수행

test_path = test_df['path'].values

# labels는 None을 입력하고 Dataset을 생성
test_ds = Plant_Dataset(
    image_filenames=test_path,
    labels=None,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    augmentor=None,
    shuffle=False,
    pre_func=xcp_preprocess_input
)

# predict()로 예측 수행
preds = xcp_model_01.predict(test_ds)

In [42]:
preds # numpy array

In [45]:
# 예측한 결과를 기반으로 별도의 결과 DataFrame을 생성

preds_df = pd.DataFrame(preds)
preds_df.columns = ['healthy', 'multiple_diseases', 'rust', 'scab']

# 테스트용 DataFrame에 바로 위에서 생성한 결과 DataFrame을 합친 뒤 이를 이용하여 submit용 DataFrame 생성
submit_df = pd.concat([test_df['image_id'], preds_df], axis=1)
submit_df.head()


In [46]:
# kaggle 제출용 csv 생성 후, kaggle에 제출 및 테스트 성능 확인

submit_df.to_csv('submit_01.csv', index=False)

In [47]:
# Image 사이즈를 변경하여 재학습 수행

# - xception 모델을 사용하되, 원본 이미지(1365, 2048)의 ratio를 어느정도 유지하면서 변경
# - 이미지 사이즈를 320, 512로 변경

from tensorflow.keras.applications.xception import preprocess_input as xcp_preprocess_input
from tensorflow.keras.applications.efficientnet import preprocess_input as eff_preprocess_input

IMAGE_SIZE = (320, 512)
BATCH_SIZE = 64

tr_path, val_path, tr_label, val_label = get_train_valid(train_df, valid_size=0.2, random_state=2021)

tr_ds = Plant_Dataset(
    tr_path,
    tr_label,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    augmentor=augmentor_01,
    shuffle=True,
    pre_func=xcp_preprocess_input
)

val_ds = Plant_Dataset(
    val_path,
    val_label,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    augmentor=None,
    shuffle=False,
    pre_func=xcp_preprocess_input
)

tr_image_batch, tr_label_batch = next(iter(tr_ds))
val_image_batch, val_label_batch = next(iter(val_ds))

print("train_image_batch shape: ", tr_image_batch.shape)
print("val_image_batch shape: ", val_image_batch.shape)
print("tr_label_batch shape: ", tr_label_batch.shape)
print("val_label_batch shape: ", val_label_batch.shape)
print(tr_image_batch[0])
print(val_image_batch[0])

In [59]:
# Config를 이용하여 학습 수행

# - 모델은 xception, image size는 (320, 512)
# - 초기 LR은 0.0001 -> LR Scheduler는 Ramp up and Step decay
# - epochs 10
# - not fine tuning
# - augmentor -> augmentor_01

from tensorflow.keras.applications.efficientnet import preprocess_input as eff_preprocess_input
from tensorflow.keras.applications.xception import preprocess_input as xcp_preprocess_input
import tensorflow as tf

# learning rate scheduler에 적용할 함수 선언
def lrfn_01(epoch):
    LR_START = 1e-5
    LR_MAX = 1e-4
    LR_RAMPUP_EPOCHS = 2
    LR_SUSTAIN_EPOCHS = 1
    LR_STEP_DECAY = 0.75
    
    def calc_fn(epoch):
        if epoch < LR_RAMPUP_EPOCHS:
            lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
        elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
            lr = LR_MAX
        else:
            lr = LR_MAX * LR_STEP_DECAY**((epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS)//2)
        return lr
    
    return calc_fn(epoch)

def lrfn_02(epoch):
    LR_START = 1e-6
    LR_MAX = 2e-5
    LR_RAMPUP_EPOCHS = 2
    LR_SUSTAIN_EPOCHS = 1
    LR_STEP_DECAY = 0.75
    
    def calc_fn(epoch):
        if epoch < LR_RAMPUP_EPOCHS:
            lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
        elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
            lr = LR_MAX
        else:
            lr = LR_MAX * LR_STEP_DECAY**((epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS)//2)
        return lr
    
    return calc_fn(epoch)

In [60]:
# Config에 입력할 callback 생성
lr01_cb = tf.keras.callbacks.LearningRateScheduler(lrfn_01, verbose=1)
lr02_cb = tf.keras.callbacks.LearningRateScheduler(lrfn_02, verbose=1)
rlr_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, mode='min', verbose=1)
ely_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)

# Augmentor 생성
augmentor_01 = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(scale_limit=(0.7, 0.9), p=0.5, rotate_limit=30),
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),
    A.Blur(p=0.2)
])

# Config 생성
class Config:
    MODEL_TYPE = 'xception'
    IMAGE_SIZE = (320, 512)
    BATCH_SIZE = 32 # 이미지 사이즈가 커졌기 때문에, 메모리 이슈로 32로 감소
    N_EPOCHS = 10 # fine tuning을 수행하지 않을 경우, 전체 수행 epoch 횟수 설정
    IS_FINE_TUNING = False
    FIRST_EPOCHS = 15 # fine tuning 일 경우, 첫 번째 epoch 횟수
    SECOND_EPOCHS = 15 # fine tuning일 경우, 두 번째 epoch 횟수
    FIRST_CALLBACKS = [lr01_cb, ely_cb] # 모델 train 시, 적용될 callback 객체 리스트
    SECOND_CALLBACKS = [lr02_cb, ely_cb] # 만일 Fine tuning 시 첫 번째 학습과 두 번째 학습의 Learning rate scheduler가 서로 다를 경우 사용
    AUGMENTOR = augmentor_01
    PRE_FUNC = xcp_preprocess_input
    INITIAL_LR = 0.0001
    DEBUG = True

In [61]:
def train_model(train_df, config=Config):
    tr_path, val_path, tr_label, val_label = get_train_valid(train_df, valid_size=0.2, random_state=2021)
    
    tr_ds = Plant_Dataset(tr_path, tr_label, image_size=config.IMAGE_SIZE, batch_size=config.BATCH_SIZE, 
                          augmentor=config.AUGMENTOR, shuffle=True, pre_func=config.PRE_FUNC)
    val_ds = Plant_Dataset(val_path, val_label, image_size=config.IMAGE_SIZE, batch_size=config.BATCH_SIZE, 
                          augmentor=None, shuffle=False, pre_func=config.PRE_FUNC)
    
    if config.DEBUG:
        tr_image_batch = next(iter(tr_ds))[0]
        val_image_batch = next(iter(val_ds))[0]
        print("train_image_batch shape: ", tr_image_batch.shape)
        print("validation_image_batch shape: ", val_image_batch.shape)
        print(tr_image_batch[0])
        print(val_image_batch[0])

    # model_type인자로 들어온 모델 생성. optimizer Adam적용. 
    print('#######', config.MODEL_TYPE, ' 생성 및 학습 수행 ########')
    model = create_model(model_type=config.MODEL_TYPE, in_shape=(config.IMAGE_SIZE[0], config.IMAGE_SIZE[1],3), n_classes=4)
    model.compile(optimizer=Adam(lr=config.INITIAL_LR), loss='categorical_crossentropy', metrics=[AUC()])

    # 만일 Fine tuning일 경우, 아래 로직 적용
    if config.IS_FINE_TUNING:
        print("########## Fine tuning 학습 시작 ##########")
        # 첫 번째 Fine Tuning
        # Feature Extractor를 제외한 classification layer를 학습.
        # Feature Extractor layer들을 trainable=False
        for layer in model.layers[:-4]:
            layer.trainable = False
            
            print("############ Classification Layer들의 학습 시작 #############")
            history = model.fit(
                    tr_ds,
                    epochs=config.FIRST_EPOCHS,
                    steps_per_epoch=int(np.ceil(tr_path.shape[0]/config.BATCH_SIZE)),
                    validation_data=val_ds,
                    validation_steps=int(np.ceil(val_path.shape[0]/config.BATCH_SIZE)),
                    callbacks=(config.FIRST_CALLBACKS),
                    verbose=1
                )
            
        # 두 번째, 전체 Layer를 학습
        # 전체 layer를 trainable=True로 수정
        # 모델이 EfficientNet 계열일 경우 Batch Normalization layer는 학습 제외
        for layer in model.layers:
            if config.MODEL_TYPE in 'efficientnet':
                if not isinstance(layer, layers.BatchNormalization):
                    layer.trainable = True
            else:
                layer.trainable = True

        print("############ 전체 Layer들의 학습을 시작 ############")
        history = model.fit(
            tr_ds,
            epochs=config.SECOND_EPOCHS,
            steps_per_epoch=int(np.ceil(tr_path.shape[0]/config.BATCH_SIZE)),
            validation_data=val_ds,
            validation_steps=int(np.ceil(val_path.shape[0]/config.BATCH_SIZE)),
            callbacks=(config.SECOND_CALLBACKS),
            verbose=1
        )
    
    else:
        print("############### 학습 시작 #################")
        history = model.fit(
            tr_ds,
            epochs=config.N_EPOCHS,
            steps_per_epoch=int(np.ceil(tr_path.shape[0]/config.BATCH_SIZE)),
            validation_data=val_ds,
            validation_steps=int(np.ceil(val_path.shape[0]/config.BATCH_SIZE)),
            callbacks=(config.FIRST_CALLBACKS),
            verbose=1
        )
    
    return model, history

In [62]:
xcp_model_02, history = train_model(train_df, config=Config)

In [65]:
# 학습된 모델을 이용하여 테스트 이미지 예측 및 결과 제출

def make_submit_df(test_df, model, config=Config):
    test_path = test_df['path'].values
    # labels는 None을 입력하고 Dataset 생성
    test_ds = Plant_Dataset(
        image_filenames=test_path,
        labels=None,
        image_size=config.IMAGE_SIZE,
        batch_size=config.BATCH_SIZE,
        augmentor=None,
        shuffle=False,
        pre_func=config.PRE_FUNC
    )
    
    # predict로 예측 수행
    preds = model.predict(test_ds)
    
    # 예측한 결과를 기반으로 별도의 결과 DataFrame을 생성
    preds_df = pd.DataFrame(preds)
    preds_df.columns = ['healthy', 'multiple_diseases', 'rust', 'scab']
    
    # 테스트용 DataFrame에 바로 위에서 생성한 결과 DataFrame을 합친 뒤, 이를 이용하여 submit용 DataFrame을 생성
    submit_df = pd.concat([test_df['image_id'], preds_df], axis=1)
    
    return submit_df

In [66]:
submit_df = make_submit_df(test_df, xcp_model_02, config=Config)

submit_df.to_csv('submit_xcp_02.csv', index=False)